In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Loading Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
# import the libraries as shown below
import tensorflow as tf
import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img



In [ ]:
#resize all the images to this
IMAGE_SIZE = [224, 224] # valid size taken by model

train_path = '/content/drive/MyDrive/Brain_Tumor_Images_Dataset/training_set'
valid_path = '/content/drive/MyDrive/Brain_Tumor_Images_Dataset/validation_set'
test_path='/content/drive/MyDrive/Brain_Tumor_Images_Dataset/test_set'

In [ ]:
#Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
from keras.applications import VGG16
vgg16= VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
vgg16.summary()

In [ ]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
from glob import glob

folders = glob('/content/drive/MyDrive/Brain_Tumor_Images_Dataset/training_set/*')

In [ ]:
folders

In [ ]:

# our layers - you can add more if you want
x = Flatten()(vgg16.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Images_Dataset/training_set',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Images_Dataset/test_set',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
val_data =  val_datagen.flow_from_directory("/content/drive/MyDrive/Brain_Tumor_Images_Dataset/validation_set",
                                            batch_size=32,
                                            target_size=(224,224),
                                            class_mode='categorical')

In [ ]:
#early stop 
early = keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=val_data,
  callbacks=[early],
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
#plot the loss
plt.plot(r.history['loss'],label='train loss')
plt.plot(r.history['val_loss'],label='val loss')

plt.legend()
plt.savefig('Loss vs val_loss')

In [ ]:
#plot the accuracy
plt.plot(r.history['accuracy'],label='train acc')
plt.plot(r.history['val_accuracy'],label='val acc')
plt.legend()
plt.savefig('AccVal_acc')

In [ ]:
#save it as a h5 file
from tensorflow.keras.models import load_model
model.save('model_vgg16.h5')

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred

In [ ]:
import numpy as np
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
y_pred

In [ ]:
from keras.preprocessing import image

In [ ]:
#model=load_model('model_vgg16.h5')

In [ ]:
#img1=image.load_img('/content/drive/MyDrive/Brain_Tumor_Images_Dataset/training_set/hemmorhage_data/030.png',target_size=(224,224))

In [ ]:
#img1

In [ ]:
img2=image.load_img('/content/drive/MyDrive/Brain_Tumor_Images_Dataset/training_set/non_hemmorhage_data/143.png',target_size=(224,224))

In [ ]:
img2

In [ ]:
x=image.img_to_array(img2)
x

In [ ]:
x.shape

In [ ]:
x=x/255

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
a=np.argmax(model.predict(img_data),axis=1)

In [ ]:
a

In [ ]:
if (a==0):
  print("hemmorhage")
else:
  print('non_hemmorhage')